In [ ]:
import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd

listing_data = []

category_list = [
    'https://www.banggood.com/Wholesale-RC-Helicopter-ca-7003.html?cat_id=7003&page=',
    'https://www.banggood.com/Wholesale-RC-Quadcopters-ca-7004.html?cat_id=7004&page=',
    'https://www.banggood.com/Wholesale-RC-Airplane-ca-7005.html?cat_id=7005&page=',
    'https://www.banggood.com/Wholesale-FPV-Racing-Drone-ca-7006.html?cat_id=7006&page=',
    'https://www.banggood.com/Wholesale-RC-Car-ca-7008.html?cat_id=7008&page='
]

def init_driver():
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("--start-maximized")
    driver = webdriver.Chrome(options=chrome_options)
    return driver

def scrape_category_page(driver, url):
    driver.get(url)
    try:
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "ul.goodlist li"))
        )
    except:
        print(f"Page did not load: {url}")
        return []

    SCROLL_PAUSE_TIME = 1.5
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(SCROLL_PAUSE_TIME)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    soup = BeautifulSoup(driver.page_source, "html.parser")
    category_tag = soup.select_one(".breadcrumb ul li:last-of-type")
    product_category = category_tag.get_text(strip=True) if category_tag else ""
    all_listings = soup.select("ul.goodlist li")
    page_data = []

    for product in all_listings:
        title_tag = product.find("a", class_="title")
        price_tag = product.find("span", class_="price")
        rating_tag = product.find("span", class_="review-text")
        review_tag = product.find("a", class_="review")

        page_data.append({
            "Title": title_tag.get_text(strip=True) if title_tag else "",
            "Price": price_tag.get_text(strip=True) if price_tag else "",
            "Rating": rating_tag.get_text(strip=True) if rating_tag else "",
            "Reviews": review_tag.get_text(strip=True) if review_tag else "",
            "Category": product_category,
            "URL": title_tag.get("href") if title_tag else ""
        })

    return page_data

def extract_data():
    driver = init_driver()
    for base_url in category_list:
        for page in range(1, 6):
            url = f"{base_url}{page}"
            print(f"Scraping: {url}")
            page_data = scrape_category_page(driver, url)
            if page_data:
                listing_data.extend(page_data)
    driver.quit()

extract_data()

df = pd.DataFrame(listing_data)
df.to_csv("banggood_products.csv", index=False)
print("Data saved to 'banggood_products.csv'")
